# Pipeline Test 

In [23]:
!pip install apache-beam[interactive]

     |████████████████████████████████| 95 kB 2.3 MB/s eta 0:00:011
  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3720 sha256=5df7ea96d1198b91d34c4634b4c314a19f181f412eb4b03d4d1f831db8e2cde6
  Stored in directory: /root/.cache/pip/wheels/c0/da/aa/ba3dfa0f64d081837addfa435f40349943bca72a79d5b08b01
Successfully built timeloop


  Attempting uninstall: httplib2
    Found existing installation: httplib2 0.19.1
    Uninstalling httplib2-0.19.1:
      Successfully uninstalled httplib2-0.19.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [35]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 187 kB 2.9 MB/s eta 0:00:01


  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.18.0
    Uninstalling google-cloud-bigquery-2.18.0:
      Successfully uninstalled google-cloud-bigquery-2.18.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [37]:
import tensorflow_model_analysis as tfma
from tfx import v1 as tfx
import os

from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
# from models import features
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from ml_metadata.proto import metadata_store_pb2

import tensorflow_data_validation as tfdv

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


## Defining Variables 

In [53]:
data_path=os.path.join('/text-classifier/Text-Classifer/ml/',"data")
data_path

'/text-classifier/Text-Classifer/ml/data'

In [54]:
context = InteractiveContext()

##  Example Gen

In [55]:
example_gen = CsvExampleGen(input_base=data_path)
example_gen

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7fdbce7b4a90>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fdbceae8c88>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

## Statistics Gen

In [56]:
statistics_gen = StatisticsGen(examples=example_genoutputs.["examples"])
statistics_gen

StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x7fdb48fc66a0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fdb48fc6eb8>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'statistics': Channel(
    type_name: ExampleStatistics
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [49]:
stats = tfdv.generate_statistics_from_csv(data_location=(data_path+'/data.csv'),delimiter=',')
stats

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


datasets {
  num_examples: 50000
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 50000
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 5000.0
          }
          buckets {
            low

## Schema Gen 

In [50]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"],infer_feature_shape=True)
schema_gen

SchemaGen(spec: <tfx.types.standard_component_specs.SchemaGenSpec object at 0x7fdad9658128>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7fdad96581d0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: SchemaGen, inputs: {'statistics': Channel(
    type_name: ExampleStatistics
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'schema': Channel(
    type_name: Schema
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [52]:
tfdv.visualize_statistics(lhs_statistics=stats)

In [51]:
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


## Example Validator

In [57]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)
example_validator

ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x7fdaaed49438>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7fdbc44ab940>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: ExampleValidator, inputs: {'statistics': Channel(
    type_name: ExampleStatistics
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'schema': Channel(
    type_name: Schema
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'anomalies': Channel(
    type_name: ExampleAnomalies
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [62]:
tfdv.validate_statistics(statistics=stats, schema=schema)

baseline {
  feature {
    name: "review"
    type: BYTES
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "sentiment"
    type: INT
    bool_domain {
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
}
anomaly_name_format: SERIALIZED_PATH